In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import ndimage, misc
from scipy.linalg import toeplitz
np.set_printoptions(threshold=np.inf)

In [92]:
G  = np.zeros((10,16))
s2 = np.sqrt(2)
#s2 = 2
for i in range(4):
    i1 = i+1
    G[i, i*4:4*i1] = 1
    G[i+4, i:16:4] = 1
G[8,0] = s2
G[9, 0:16:5] = s2 

#print(G)

In [115]:
[u, s, v] = np.linalg.svd(G)
#print(s)

Cm = s[0]/s[-1]
print(Cm)

p = 9
u = u[0:p, 0:p]
v = v[0:p, 0:p]
s = s[0:p]

Cm = s[0]/s[-1]
print(Cm)

6651738713120766.0
4.625150399967511
